In [27]:
from openai import OpenAI
import re
client = OpenAI(
  api_key="sk-b8fbb063539a4a4d825049c0b12cb390",
  base_url="https://api.deepseek.com/v1"
)

In [28]:
DEFAULT_MODEL="deepseek-chat"

In [22]:
# response = client.chat.completions.create(
#     model="deepseek-chat",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant"},
#         {"role": "user", "content": "Hell0"}
#     ],
#     stream=False
# )
# print(response.choices[0].message.content)

Hello! How can I assist you today? 😊


In [29]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def invoke(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model =  DEFAULT_MODEL,
            messages= self.messages,
            temperature= 0
        )
        return completion.choices[0].message.content

In [30]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

ask_fruit_unit_price:
e.g. ask_fruit_unit_price: apple
Asks the user for the price of a fruit

Example session:

Question: What is the unit price of apple?
Thought: I need to ask the user for the price of an apple to provide the unit price. 
Action: ask_fruit_unit_price: apple
PAUSE

You will be called again with this:

Observation: Apple unit price is 10/kg

You then output:

Answer: The unit price of apple is 10 per kg.
""".strip()

In [39]:
def calculate(what):
    return eval(what)

def ask_fruit_unit_price(fruit):
    if fruit.casefold() == "apple":
        return "Apple unit price is 10/kg"
    elif fruit.casefold() == "banana":
        return "Banana unit price is 6/kg"
    else: return "{} unit price is 20/kg".format(fruit)

In [40]:
action_re=re.compile(r'^Action: (\w+): (.*)$')
known_actions = {
    "calculate": calculate, 
    "ask_fruit_unit_price": ask_fruit_unit_price
}

def query(question, max_turn=5):
    i=0
    agent=Agent(prompt)
    next_prompt=question
    while i< max_turn:
        i += 1
        result = agent.invoke(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                return Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [41]:
query("What is the total price of 3kg of apple and 2kg of banana?")

Thought: To calculate the total price, I need to know the unit prices of both apples and bananas. I will ask the user for the unit price of apples and bananas first.
Action: ask_fruit_unit_price: apple
PAUSE
 -- running ask_fruit_unit_price apple
Observation: Apple unit price is 10/kg
Thought: I have the unit price of apples (10/kg). Now I need the unit price of bananas to proceed with the calculation.
Action: ask_fruit_unit_price: banana
PAUSE
 -- running ask_fruit_unit_price banana
Observation: Banana unit price is 6/kg
Thought: I now have the unit prices for both apples (10/kg) and bananas (6/kg). I will calculate the total price for 3kg of apples and 2kg of bananas.
Action: calculate: (3 * 10) + (2 * 6)
PAUSE
 -- running calculate (3 * 10) + (2 * 6)
Observation: 42
Answer: The total price of 3kg of apples and 2kg of bananas is 42.
